# Prism Classifier
1. Construa um classificador PRISM em Python
2. Aplicar o algoritmo a um dataset teste

## Resolução do exercício
### Imports



In [ ]:
import numpy as np
import pandas as pd
from dfply import *

### Algoritmo Prism

In [18]:
def compute_probabilities(df, target):
    prob_table = pd.DataFrame()
    feature_matrix = df.drop(target, axis=1)
    feature_variables = feature_matrix.columns.values
    target_variable = target

    for feature_variable in feature_variables:
        for unique_value in feature_matrix[feature_variable].unique():
            prob_table = prob_table.append((df >> mask(X[feature_variable] == unique_value)
                                               >> group_by(X[target_variable])
                                               >> summarize(count = n(X[feature_variable]))
                                               >> ungroup()
                                               >> rename(target = X[target_variable])
                                               >> mutate(probability = X['count']/X['count'].sum())
                                               >> mutate(variable = feature_variable)
                                               >> mutate(value = unique_value)
                                               >> select(['target', 'variable', 'value', 'probability'])), ignore_index=True)
    return prob_table

def return_best_rule(prob_table):
    best_rule = prob_table >> mask(X['probability'] == X['probability'].max())
    return best_rule.iloc[0, :]

def update_table(prob_table, rule):
    for idx in prob_table.index:
        if prob_table.iloc[idx,:].equals(rule):
            new_table = prob_table.drop(idx).reset_index(drop=True)
    return new_table

def PRISM(df, target):
    rules = []
    prob_table = compute_probabilities(df, target)
    while prob_table.empty == False:
        best_rule = return_best_rule(prob_table)
        rules.append(best_rule['variable']+' == '+best_rule['value'])
        prob_table = update_table(prob_table, best_rule)
    return rules

In [19]:
#Criando dados fictícios - Exemplo das aulas
data = {'height': ['short', 'short', 'tall', 'tall', 'tall', 'tall', 'tall', 'short'],
        'hair': ['blond', 'blond', 'red', 'dark', 'dark', 'blond', 'dark', 'blond'],
        'teste': ['C1', 'C2', 'C1', 'C2', 'C2', 'C1', 'C2', 'C2']}

data2 = {'Outlook': ['Overcast', 'Overcast', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Rain', 'Rain','Overcast','Rain','Sunny','Sunny','Rain','Sunny'],
        'Temperature': ['Hot', 'Mild', 'Mild', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Hot', 'Cool', 'Hot', 'Hot', 'Mild', 'Mild'],
        'Humidity': ['Normal', 'High', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'High','High','Normal','High','High','High','High'],
        'Wind':['Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Strong', 'Weak', 'Weak','Weak', 'Strong', 'Strong', 'Weak', 'Strong', 'Weak' ],
        'Play' :['Yes','Yes','Yes','Yes','Yes','Yes','Yes','Yes','Yes','No','No','No','No','No']}

#Transformando em DataFrame
df = pd.DataFrame(data2)

#Chamando a função do algoritmo PRISM no DataFrame e indicando o nome da coluna da variável alvo (classes).
rules = PRISM(df, 'Play')

#Imprimindo a lista de regras associativas, que por padrão já estão em ordem crescente de importância.
print(rules)

['Outlook == Overcast', 'Humidity == Normal', 'Temperature == Cool', 'Wind == Weak', 'Temperature == Mild', 'Outlook == Sunny', 'Outlook == Rain', 'Humidity == High', 'Temperature == Hot', 'Temperature == Hot', 'Wind == Strong', 'Wind == Strong', 'Humidity == High', 'Outlook == Sunny', 'Outlook == Rain', 'Temperature == Mild', 'Temperature == Cool', 'Wind == Weak', 'Humidity == Normal']
